# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [8]:
import pandas as pd
import numpy as np 
import xlsxwriter 
import requests
import math
from scipy import stats


## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [9]:
from secrets import IEX_CLOUD_SANDBOX_API_TOKEN
stocks_df = pd.read_csv('sp_500_stocks.csv')
stocks_df.columns


Index(['Ticker'], dtype='object')

## Making Our First API Call

It's now time to make the first version of our momentum screener!

Retrieve price and one-year stock returns

In [10]:
base_url="https://sandbox.iexapis.com"
params={"token":IEX_CLOUD_SANDBOX_API_TOKEN}
response = requests.get(f"{base_url}/stable/stock/fb/stats", params=params)
response.json()
# "GET /stock/{symbol}/stats/{stat?}"
# find price and one year stock returns in batch, lets do simple fisrt


{'companyName': 'Facebook Inc',
 'marketcap': 911251481408,
 'week52high': 341.73,
 'week52low': 212.04,
 'week52highSplitAdjustOnly': 341.94,
 'week52lowSplitAdjustOnly': 214.1,
 'week52change': 0.4785625414643297,
 'sharesOutstanding': 2495971328,
 'float': 0,
 'avg10Volume': 20245867,
 'avg30Volume': 21418766,
 'day200MovingAvg': 285.05,
 'day50MovingAvg': 317.9,
 'employees': 55102,
 'ttmEPS': 11.94,
 'ttmDividendRate': 0,
 'dividendYield': 0,
 'nextDividendDate': '0',
 'exDividendDate': '0',
 'nextEarningsDate': '2021-07-13',
 'peRatio': 25.973091887269092,
 'beta': 1.2763155859615456,
 'maxChangePercent': 7.103284861490907,
 'year5ChangePercent': 1.6041090941699254,
 'year2ChangePercent': 0.7082018622695044,
 'year1ChangePercent': 0.5079313474188858,
 'ytdChangePercent': 0.11970530627472227,
 'month6ChangePercent': 0.10275230070425938,
 'month3ChangePercent': 0.13404180779063526,
 'month1ChangePercent': -0.01514581208280194,
 'day30ChangePercent': -0.01491332011965066,
 'day5Chan

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [11]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   

    
symbol_groups = list(chunks(stocks_df['Ticker'], 100))
symbol_strings = []


for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
   
    
#     /stock/market/batch?symbols=aapl,fb,tsla&types=quote,news,chart&range=1m&last=5
#     print(symbol_strings[i]) #string batches
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
params={"token":IEX_CLOUD_SANDBOX_API_TOKEN}
requests.get(f"{base_url}/stable/stock/ETFC/stats",params=params)
response.json()


{'companyName': 'Facebook Inc',
 'marketcap': 911251481408,
 'week52high': 341.73,
 'week52low': 212.04,
 'week52highSplitAdjustOnly': 341.94,
 'week52lowSplitAdjustOnly': 214.1,
 'week52change': 0.4785625414643297,
 'sharesOutstanding': 2495971328,
 'float': 0,
 'avg10Volume': 20245867,
 'avg30Volume': 21418766,
 'day200MovingAvg': 285.05,
 'day50MovingAvg': 317.9,
 'employees': 55102,
 'ttmEPS': 11.94,
 'ttmDividendRate': 0,
 'dividendYield': 0,
 'nextDividendDate': '0',
 'exDividendDate': '0',
 'nextEarningsDate': '2021-07-13',
 'peRatio': 25.973091887269092,
 'beta': 1.2763155859615456,
 'maxChangePercent': 7.103284861490907,
 'year5ChangePercent': 1.6041090941699254,
 'year2ChangePercent': 0.7082018622695044,
 'year1ChangePercent': 0.5079313474188858,
 'ytdChangePercent': 0.11970530627472227,
 'month6ChangePercent': 0.10275230070425938,
 'month3ChangePercent': 0.13404180779063526,
 'month1ChangePercent': -0.01514581208280194,
 'day30ChangePercent': -0.01491332011965066,
 'day5Chan

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [15]:
final_df = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
  

    params={"symbols":symbol_string, "types":"stats,price", "token":IEX_CLOUD_SANDBOX_API_TOKEN}
    response = requests.get(f"{base_url}/stable/stock/market/batch", params=params)
    print(response.status_code)
    data = response.json()
    if "ETFC" in data:
        print(data["ETFC"])

    for ticker in symbol_string.split(','):
        row = pd.Series([
            ticker,
            data[ticker]['price'],
            data[ticker]['stats']['year1ChangePercent'],
            "N/A"
            ], index=my_columns)
        final_df = final_df.append(row, ignore_index=True)
      
    #         if row['Ticker'] == 'EFTC':
    #             print(row)
            
        
#         print(final_df.loc[final_df['Ticker'] == ticker].iloc[0]['One-Year Price Return'])
        one_year_pr = final_df.loc[final_df['Ticker'] == ticker].iloc[0]['One-Year Price Return']
        assert(one_year_pr == data[ticker]['stats']['year1ChangePercent'])

        


200
200
{'stats': {'companyName': 'E*TRADE Financial Corp.', 'marketcap': 11040851060, 'week52high': 58.8, 'week52low': 26.91, 'week52highSplitAdjustOnly': None, 'week52lowSplitAdjustOnly': None, 'week52change': 39.1226, 'sharesOutstanding': 231444314, 'float': 230534844, 'avg10Volume': 3724348, 'avg30Volume': 2682050.4, 'day200MovingAvg': 46.07, 'day50MovingAvg': 54.58, 'employees': 4123, 'ttmEPS': 3.4946, 'ttmDividendRate': 0.56, 'dividendYield': None, 'peRatio': None, 'beta': None, 'maxChangePercent': None, 'year5ChangePercent': None, 'year2ChangePercent': None, 'year1ChangePercent': None, 'ytdChangePercent': None, 'month6ChangePercent': None, 'month3ChangePercent': None, 'month1ChangePercent': None, 'day30ChangePercent': None, 'day5ChangePercent': None}, 'price': 49.91}
200
200
200
200


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.

Steps
- Sort stocks by one price year return change
- drop the bottom 450 stocks


In [16]:

final_df.describe()
final_df.sort_values(by=['One-Year Price Return'], ascending=False, inplace=True)
final_df = final_df[:50]
final_df.reset_index(inplace=True)

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [19]:
def get_portfolio():
    portfolio = input("What is the size of your portfolio? ")
    portfolio = float(portfolio)
    return portfolio
portfolio_size

1000000.0

In [20]:
portfolio = get_portfolio()

What is the size of your portfolio? 1000000


In [21]:
position_per_stock = portfolio/len(final_df.index)

for i in range(len(final_df.index)):
    shares_to_buy = math.floor(position_per_stock/final_df.loc[i, 'Price'])
    final_df.at[i, 'Number of Shares to Buy'] = shares_to_buy
    
    
final_df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,70.93,5.461375,281
1,179,FCX,42.32,4.038062,472
2,208,GPS,37.38,3.985978,535
3,253,IVZ,29.13,3.305125,686
4,441,TPR,47.42,2.854918,421
5,272,KSS,63.58,2.611306,314
6,410,SIVB,600.17,2.510994,33
7,314,MOS,37.06,2.46312,539
8,129,DFS,117.44,2.30057,170
9,385,PWR,100.59,2.236686,198


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [117]:
hqm_colums = ["Ticker", "Price", "Number of Shares to Buy",
              "One-Year Price Return", "One-Year Return Percentile",
              "Six-Month Price Return", "Six-Month Return Percentile",
              "Three-Month Price Return", "Three-Month Return Percentile", 
              "One-Month Price Return", "One-Month Return Percentile", "HQM Score"]

hqm_df = pd.DataFrame(columns=hqm_colums)
for symbol_string in symbol_strings:
  
    params={"symbols":symbol_string, "types":"stats,price", "token":IEX_CLOUD_SANDBOX_API_TOKEN}
    response = requests.get(f"{base_url}/stable/stock/market/batch", params=params)
    symbol_list = symbol_string.split(',')
#     print(response.status_code)
    
    for ticker in symbol_list:
        ticker_data = response.json()[ticker]
#         print(ticker_data)
        row = pd.Series([
            ticker,
            ticker_data['price'],
            "N/A",
            ticker_data['stats']['year1ChangePercent'],
            "N/A",
               ticker_data['stats']['month6ChangePercent'],
            "N/A",
            ticker_data['stats']['month3ChangePercent'],
            'N/A',
            ticker_data['stats']['month1ChangePercent'],
            "N/A",
            "N/A"
        ], index=hqm_colums)
        hqm_df = hqm_df.append(row, ignore_index=True)
   

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [118]:
time_periods = [
    'One-Month',
    'Six-Month',
    'Three-Month',
    'One-Year',
]

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [119]:
#Calculate percentile score using scipy
for tp in time_periods:
    hqm_df[f"{tp} Price Return"].fillna(0, inplace=True)
for i in range(len(stocks_df.index)):
    for tp in time_periods:
        p_label = f"{tp} Return Percentile"
        pr_label = f"{tp} Price Return"
        
        price_return = hqm_df.loc[i, pr_label]
        
        hqm_df.at[i, p_label] = stats.percentileofscore(hqm_df[pr_label], price_return)/100
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,133.34,N/A,0.654081,0.540594,0.186953,0.467327,0.017848,0.241584,-0.022932,0.239604,N/A
1,AAL,23.50,N/A,1.344547,0.869307,0.744264,0.956436,0.238362,0.861386,-0.060275,0.144554,N/A
2,AAP,209.59,N/A,0.740804,0.59802,0.346961,0.706931,0.343788,0.970297,0.095248,0.851485,N/A
3,AAPL,127.58,N/A,0.660733,0.546535,0.051998,0.19802,-0.078882,0.09505,-0.069589,0.122772,N/A
4,ABBV,117.62,N/A,0.386097,0.320792,0.214734,0.532673,0.130975,0.548515,0.093768,0.845545,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,120.39,N/A,0.516528,0.435644,0.165078,0.445545,0.122256,0.526733,0.005699,0.378218,N/A
501,ZBH,169.91,N/A,0.517344,0.439604,0.100497,0.30297,0.028027,0.265347,-0.017203,0.263366,N/A
502,ZBRA,490.45,N/A,1.152316,0.80198,0.376799,0.742574,-0.012361,0.174257,-0.065854,0.132673,N/A
503,ZION,62.70,N/A,1.375917,0.881188,0.606907,0.912871,0.213430,0.8,0.071226,0.730693,N/A


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

hqm score is the mean of a stock is the average of all percentile scores

In [121]:
from statistics import mean

for row in hqm_df.index:
    percentile_arr = []
    for col in hqm_df:
        if "Percentile" in col:
            percentile_arr.append(hqm_df.loc[row, col])
        
    hqm_score = mean(percentile_arr)
    hqm_df.at[row, "HQM Score"] = hqm_score
hqm_df.sort_values(by=["HQM Score"], inplace=True, ascending=False)
hqm_df = hqm_df[:50]
hqm_df.reset_index(inplace=True, drop=True)
hqm_df

<ipython-input-121-190e197f883a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_df.sort_values(by=["HQM Score"], inplace=True, ascending=False)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,42.95,N/A,4.028933,0.99802,1.140765,0.992079,0.350232,0.974257,0.258340,0.994059,0.989604
1,NUE,103.86,N/A,1.862332,0.958416,1.013338,0.988119,0.831639,1.0,0.295843,1.0,0.986634
2,COF,164.45,N/A,1.954645,0.968317,0.839866,0.976238,0.334794,0.960396,0.185139,0.970297,0.968812
3,KSU,312.34,N/A,1.370501,0.879208,0.705067,0.944554,0.510714,0.99802,0.212412,0.984158,0.951485
4,DXC,38.46,N/A,1.435835,0.90495,0.754187,0.960396,0.385787,0.988119,0.144818,0.940594,0.948515
5,TXT,70.03,N/A,1.888825,0.962376,0.557432,0.883168,0.311368,0.940594,0.158609,0.954455,0.935149
6,WFC,48.42,N/A,1.119475,0.792079,0.976183,0.986139,0.389385,0.990099,0.172859,0.964356,0.933168
7,WRK,63.29,N/A,1.615350,0.936634,0.513207,0.853465,0.379151,0.984158,0.160229,0.958416,0.933168
8,FITB,44.64,N/A,1.807770,0.954455,0.683435,0.936634,0.308747,0.938614,0.102585,0.875248,0.926238
9,CINF,121.17,N/A,1.614415,0.934653,0.566034,0.885149,0.316290,0.942574,0.141586,0.938614,0.925248


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [122]:

portfolio = get_portfolio()

What is the size of your portfolio? 1000000


In [123]:
position_size = portfolio/len(hqm_df.index)

for i in hqm_df.index:
    num_shares = math.floor(position_size / hqm_df.loc[i, "Price"])
    hqm_df.at[i, "Number of Shares to Buy"] = num_shares
#     print(hqm_df.loc[i, "index"])
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,42.95,465,4.028933,0.99802,1.140765,0.992079,0.350232,0.974257,0.258340,0.994059,0.989604
1,NUE,103.86,192,1.862332,0.958416,1.013338,0.988119,0.831639,1.0,0.295843,1.0,0.986634
2,COF,164.45,121,1.954645,0.968317,0.839866,0.976238,0.334794,0.960396,0.185139,0.970297,0.968812
3,KSU,312.34,64,1.370501,0.879208,0.705067,0.944554,0.510714,0.99802,0.212412,0.984158,0.951485
4,DXC,38.46,520,1.435835,0.90495,0.754187,0.960396,0.385787,0.988119,0.144818,0.940594,0.948515
5,TXT,70.03,285,1.888825,0.962376,0.557432,0.883168,0.311368,0.940594,0.158609,0.954455,0.935149
6,WFC,48.42,413,1.119475,0.792079,0.976183,0.986139,0.389385,0.990099,0.172859,0.964356,0.933168
7,WRK,63.29,316,1.615350,0.936634,0.513207,0.853465,0.379151,0.984158,0.160229,0.958416,0.933168
8,FITB,44.64,448,1.807770,0.954455,0.683435,0.936634,0.308747,0.938614,0.102585,0.875248,0.926238
9,CINF,121.17,165,1.614415,0.934653,0.566034,0.885149,0.316290,0.942574,0.141586,0.938614,0.925248


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [128]:
writer = pd.ExcelWriter(engine='xlsxwriter', path='recommmended_momentum_stocks.xlsx')
hqm_df.to_excel(writer, sheet_name="Momentum", index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [129]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [130]:
hqm_columns = hqm_colums

workbook = xlsxwriter.Workbook("recommmended_momentum_stocks.xlsx")

for i in range(len(hqm_columns)):
    excel_col = chr(ord("A") + i)
    ws = writer.sheets["Momentum"]
    if "Ticker" == hqm_columns[i]:
        ws.set_column(f"{excel_col}:{excel_col}", 20, cell_format=string_template)
    if "Percentile" in hqm_columns[i] or "Return" in hqm_columns[i] or "Score" in hqm_columns[i]:
      
        ws.set_column(f"{excel_col}:{excel_col}", 20,cell_format=percent_template)
        
    if "Shares" in hqm_columns[i]:
         ws.set_column(f"{excel_col}:{excel_col}", 20, cell_format=integer_template)
    if "Price" == hqm_columns[i]:
         ws.set_column(f"{excel_col}:{excel_col}", 20, cell_format=dollar_template)
        
    ws.write_column(f"{excel_col}0", hqm_df[hqm_columns[i]])
    
    


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [131]:
writer.save()